# LlamaIndex framework
>LlamaIndex is the leading data framework for building LLM applications

## 1. Installation

### Default installation for OpenAI integration

```bash
pip install llama-index
```
Which includes the following packages:
- llama-index-core
- llama-index-llms-openai
- llama-index-embeddings-openai
- llama-index-program-openai
- llama-index-question-gen-openai
- llama-index-agent-openai
- llama-index-readers-file
- llama-index-multi-modal-llms-openai

**LLAMA_INDEX_CACHE_DIR** - Environment variable to set the cache directory

### Custom installation for local LLMs

```bash
pip install llama-index-core llama-index-readers-file llama-index-llms-ollama llama-index-embeddings-huggingface
```
Which can use local LLMs from Ollama, and indexing with Huggingface

## 2. Load Data and build an index

In [ ]:
# !pip install llama-index
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# Load data from the directory
documents = SimpleDirectoryReader("data").load_data()
print(documents)
# Load specific file type, i.e. ".pdf"
documents1 = SimpleDirectoryReader("data", required_exts=[".pdf"]).load_data()
print(documents1)
# Load specific files, which `input_files` will override `input_dir`.
documents2 = SimpleDirectoryReader(input_dir="data", input_files=["/path/to/data/requirements.txt"]).load_data()
print(documents2)

# Build index
index = VectorStoreIndex.from_documents(documents)
print(index)

## 3. Query your data



In [ ]:
query_engine = index.as_query_engine()

# Query
response = query_engine.query("`688396.SH`是哪家公司?")
print(response)

## 4. Logging - Queries and Events

In [ ]:
import logging, sys

logging.basicConfig(stream=sys.stdout, level=logging.WARN)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## 5. Storing your index

By default, the data you just loaded is stored in memory as a series of vector embeddings.
You can save time (and requests to OpenAI) by saving the embeddings to disk. That can be done with this line:

In [ ]:
index.storage_context.persist()

Of course, you don't get the benefits of persisting unless you load the data. So let's modify the code to generate and store the index if it doesn't exist, but load it if it does:

In [ ]:
import os.path
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage

PERSIST_DIR = "./storage"

print(os.path.getsize(PERSIST_DIR))

if os.path.exists(PERSIST_DIR) and os.path.isdir(PERSIST_DIR) and any(os.scandir(PERSIST_DIR)):
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)
else:
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir=PERSIST_DIR)

query_engine = index.as_query_engine()
response = query_engine.query("`688396.SH`是哪家公司?")
print(response)

## 6. Using local LLM instead of OpenAI API

### Start Ollama serve with a chosen LLM model

```bash
# Run Ollama serve
docker run -d -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama

# Exec a given LLM model
docker exec -it ollama ollama run deepseek-r1:7b
```

### Load data and build an index with local LLM, and Query

In [ ]:
# !pip install llama-index-core llama-index-readers-file llama-index-llms-ollama llama-index-embeddings-huggingface
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama

documents = SimpleDirectoryReader("data").load_data()
Settings.embed_model = HuggingFaceEmbedding("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")
Settings.llm = Ollama("deepseek-r1:7b", request_timeout=360.0)

index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()
response = query_engine.query("`688396.SH`是哪家公司?")
print(response)

DEBUG:llama_index.core.readers.file.base:> [SimpleDirectoryReader] Total files added: 3
> [SimpleDirectoryReader] Total files added: 3
> [SimpleDirectoryReader] Total files added: 3
DEBUG:fsspec.local:open file: /Users/zcui/Workspace/github/llm-projects/week8-llama-index/data/README_zh-CN.md
open file: /Users/zcui/Workspace/github/llm-projects/week8-llama-index/data/README_zh-CN.md
open file: /Users/zcui/Workspace/github/llm-projects/week8-llama-index/data/README_zh-CN.md
DEBUG:fsspec.local:open file: /Users/zcui/Workspace/github/llm-projects/week8-llama-index/data/pdf内容研报.pdf
open file: /Users/zcui/Workspace/github/llm-projects/week8-llama-index/data/pdf内容研报.pdf
open file: /Users/zcui/Workspace/github/llm-projects/week8-llama-index/data/pdf内容研报.pdf
DEBUG:fsspec.local:open file: /Users/zcui/Workspace/github/llm-projects/week8-llama-index/data/requirements.txt
open file: /Users/zcui/Workspace/github/llm-projects/week8-llama-index/data/requirements.txt
open file: /Users/zcui/Workspace/gi

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]